In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
df = pd.read_csv('lwe_thickness_time_series.csv')
#df

In [ ]:
# Extracting the lon-lat from the dataframe and droping the columns from the extracted TWS
lat_lon_df = df[['lat', 'lon']]
time_series_df = df.drop(columns=['lat', 'lon'])

In [ ]:
#RETIMING THE DATA FRAME.

# Transposed the DataFrame
time_series_df = time_series_df.T

# Converted column names to datetime
time_series_df.index = pd.to_datetime(time_series_df.index)

# Custom function to calculate average for months with more than one reading
# or keep the first non-null value for months with only one reading
def custom_resampler(array_like):
    non_null_values = array_like.dropna()
    if len(non_null_values) > 1:
        return np.mean(non_null_values)
    elif len(non_null_values) == 1:
        return non_null_values.iloc[0]
    else:
        return np.nan

# Resampled the DataFrame using the custom function
time_series_df = time_series_df.resample('MS').apply(custom_resampler)

# Transposed the DataFrame back
time_series_df = time_series_df.T

# Interpolated missing values
time_series_df.interpolate(method='linear', axis=1, limit_direction='both', inplace=True)

In [ ]:
# Combined the lat/lon columns and the time series data
result_df = pd.concat([lat_lon_df, time_series_df], axis=1)

In [ ]:
# Made sure column names are formatted as strings
result_df.columns = result_df.columns.astype(str)

In [ ]:
result_df

In [ ]:
result_df.to_csv('lwe_thickness_retimed.csv', index=False)